In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt
from collections import namedtuple
import matplotlib.patches as patches
import networkx as nx

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/19-Table 1.csv')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/30-Table 1.csv')
# df = pd.read_csv('/Users/bryan/Studio/lcmsseq/examples/compounds.csv', skiprows=2)
df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/5mix.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/Ning1024/2mix_180521s14_100.xls')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/Ning1024/12mixtures/12mix.csv')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724_std.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/oligosdegradsample2-1.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_component_only.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx')
# df = thermo_df(df)
# df = df[df.Mass < 20000]
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/032919s07.xls')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/111418s05.xls')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/100918s06.xls')
df.shape
plot_zone(df)

In [ ]:
df_sample = df[(df.Mass < 8000) & (df.RT < 11)]
plot_zone(df_sample)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
# df_hi = df_chosen
df_lo = df_chosen

In [ ]:
x = df_hi.Mass
y = df_hi.RT
z = np.polyfit(x, y, 4)
def f_hi2(x):
    return z[0] * x**4 + z[1]*x**3 + z[2]*x**2 + z[3]*x + z[4] + 0.1
y = f_hi2(x)
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.plot(x, y, "r-")

plt.scatter(x, y)
plt.show()

In [ ]:
x = df_lo.Mass
y = df_lo.RT
z_lo = np.polyfit(x, y, 4)
def f_lo2(x):
    return z_lo[0] * x**4 + z_lo[1]*x**3 + z_lo[2]*x**2 + z_lo[3]*x + z_lo[4] - 0.1
#     return z_lo[0] * x**3 + z_lo[1]*x**2 + z_lo[2]*x + z_lo[3]
y = f_lo2(x)
plt.figure(figsize=(16, 12))
plt.plot(x, y, "r-")
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(x, y)
plt.show()

In [ ]:
_, ax = plt.subplots(figsize=(16, 12))
ax.scatter(df_sample.Mass, df_sample.RT)

ax.scatter(df_hi.Mass, df_hi.RT, c='blue')
ax.plot(df_hi.Mass, f_hi2(df_hi.Mass), c='blue')

ax.scatter(df_lo.Mass, df_lo.RT, c='green')
ax.plot(df_lo.Mass, f_lo2(df_lo.Mass), c='green')

In [ ]:
sample_rt_min = df_sample.RT.min()
sample_rt_max = df_sample.RT.max()

In [ ]:
filter_min = 0.5
filter_nums = 5

def bin_range2(df, left, right, f_hi, f_lo, bin_num):
    rt = df.RT
    if rt.empty:
        return 0.0, 0.0
#     rt_min = rt.min()
#     rt_max = rt.max()
    rt_min = f_lo(left)
    rt_min = sample_rt_min if rt_min < sample_rt_min else rt_min
    rt_max = f_hi(right)
    rt_max = sample_rt_max if rt_max > sample_rt_max else rt_max
    rt_height = rt_max - rt_min
#     print(rt_min, rt_max, rt_height)
    
    bin_size = rt_height / filter_nums
    bin_size = filter_min if bin_size < filter_min else bin_size
    
    rt_hi = rt_max - bin_size * (bin_num - 1)
    rt_lo = rt_hi - bin_size
#     print(type(rt_lo), type(bin_size))
    if np.less(rt_lo, rt_min):
        print('Reach the last bin.')
        rt_lo = rt_min
        rt_hi = rt_lo + bin_size
    print('df_win count {} winRT {}-{} filterRT lo-hi {}-{}'.format(df.shape[0], rt_min, rt_max, rt_lo, rt_hi))
    return rt_lo, rt_hi

In [ ]:
def create_rectangle(left, right, lo, hi):
    rect = patches.Rectangle((left, lo), right-left, hi-lo)
    return rect

In [ ]:
cursor = 0
mass_win_size = 10
bin_num = 3
last_rt_lo = 0.0

# plt.figure(figsize=(16, 9))
fig, ax = plt.subplots(1, figsize=(16, 9))
ax.scatter(df.Mass, df.RT)
filters = list()
while cursor < df_sample.Mass.max():
    # do sth
#     print('processing win {}-{}'.format(cursor, cursor+mass_win_size))
    df_win = df[(df.Mass >= cursor) & (df.Mass < cursor + mass_win_size)]
    rt_lo, rt_hi = bin_range2(df_win, cursor, cursor+mass_win_size, f_hi2, f_lo2, bin_num)
    if rt_lo < last_rt_lo:
        rt_lo = last_rt_lo
        if rt_lo >= rt_hi:
            cursor += mass_win_size
            continue
            
    if np.isclose(rt_lo, 0.0) or np.isclose(rt_hi, 0.0):
        cursor += mass_win_size
        continue
    
    rect = create_rectangle(cursor, cursor+mass_win_size, rt_lo, rt_hi)
    ax.add_patch(rect)
    
    last_rt_lo = rt_lo
    df_filter = df_win[(df_win.RT>=rt_lo) & (df_win.RT<=rt_hi)]
#     print(cursor, df_filter.shape)
    filters.append(df_filter)
    cursor += mass_win_size

plt.show()

In [ ]:
df_filters = pd.concat(filters)
plt = plot_zones(df_sample, df_filters)
plt.plot(df_hi.Mass, f_hi2(df_hi.Mass))
plt.plot(df_lo.Mass, f_lo2(df_lo.Mass))
plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/{}/filter_{}_bin_{}_filters_{}.png'.format('mix5', filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
bcr_5p = base_calling_random(df_filters)
plt, _ = plot_basecalling(*bcr_5p, annotate=False)
plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/{}/filter_bc_{}_bin_{}_filters_{}.png'.format('mix5', filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
G = nx.DiGraph()

df_bcr = bcr_5p[0]
mass_pairs = bcr_5p[1]
for t in mass_pairs:
    G.add_edge(t[1], t[0], base=t[-1])

path = longest_path(G)
seqs = [G.edges[path[i], path[i+1]]['base'] for i in range(len(path)-1)]
print('longest sequence: ' + ''.join(seqs))
print(path)

In [ ]:
def longest_path(G):
    dist = {} # stores [node, distance] pair
    for node in nx.topological_sort(G):
        # pairs of dist,node for all incoming edges
        pairs = [(dist[v][0]+1,v) for v in G.pred[node]] 
        if pairs:
            dist[node] = max(pairs)
        else:
            dist[node] = (0, node)
    node,(length,_)  = max(dist.items(), key=lambda x:x[1])
    path = []
    while length > 0:
        path.append(node)
        length,node = dist[node]
    return list(reversed(path))

# Mass Sum

In [ ]:
df_src = df_sample.sort_values('Vol', ascending=False)[:2000]

In [ ]:
plot_zone(df_src)

In [ ]:
full_mass = 6781.0694
df_3p_cca, df_5p_cca = computational_data_seperation(df_src, df_src, full_mass, ignore_endpoints=False, tag=694.2397)

In [ ]:
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(df_3p_cca.Mass, df_3p_cca.RT)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_sample = df[df.Mass < 20000]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
# df_3p = df_chosen
df_5p = df_chosen

In [ ]:
full_mass = 7088.0855 #7079.0852 #6781.0694
df_3p_cca, df_5p_cca = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True, tag=694.2397)

In [ ]:
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.scatter(df_3p_cca.Mass, df_3p_cca.RT)
plt.scatter(df_5p_cca.Mass, df_5p_cca.RT)